In [2]:
import json
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import LSTM, Dense

# Load dos dados do ficheiro JSON e preparação dos dados para treino e teste
def load_dataset(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)

    captured_data = data['capturedData']

    accelerometerXAxis = [entry['accelerometerXAxis'] for entry in captured_data]
    accelerometerYAxis = [entry['accelerometerYAxis'] for entry in captured_data]
    accelerometerZAxis = [entry['accelerometerZAxis'] for entry in captured_data]
    gyroscopeXAxis = [entry['gyroscopeXAxis'] for entry in captured_data]
    gyroscopeYAxis = [entry['gyroscopeYAxis'] for entry in captured_data]
    gyroscopeZAxis = [entry['gyroscopeZAxis'] for entry in captured_data]

    # criar dataFrame 
    df = pd.DataFrame({
        'accelerometerX_LeftRight': accelerometerXAxis,
        'accelerometerY_FrontBack': accelerometerYAxis,
        'accelerometerZ_Vertical': accelerometerZAxis,
        'gyroscopeXAxis': gyroscopeXAxis,
        'gyroscopeYAxis': gyroscopeYAxis,
        'gyroscopeZAxis': gyroscopeZAxis
    })
    
    return df

def classify_acceleration(data):
    SPEED = 27.7778  # 100 km/h in m/s
    
    # Calculate Acceleration
    ## accel = SPEED/time
    accel = SPEED / 5.5  # 5.5 seconds to reach 100 km/h
    
    # Calculate g-Force
    # g = accel / 9.81
    gforce = accel / 9.81    

    # critérios para classificação 
    slow_threshold = 0.3 * gforce
    aggressive_threshold = gforce
    # print all in one line with gforce
    print(f"Acceleration: {accel}, g-force: {gforce}, slow threshold: {slow_threshold}, aggressive threshold: {aggressive_threshold}")

    # funcao para classificar a aceleração
    def classify(gforce):
        if gforce >= aggressive_threshold:
            return 'aggressive'
        elif gforce >= slow_threshold:
            return 'normal'
        else:
            return 'slow'

    # aplicar a função de classificação
    data['classification'] = data['accelerometerY_FrontBack'].apply(classify)
        
            
    print(data)

    return data


def classify(data):
    
    # 4by4, accelerometerY sum to classify
    ###### check thresholds
    
    for i in range(0, len(data), 4):
        sum = 0
        for j in range(0, 3):
            sum += data['accelerometerY_FrontBack'][i+j]
            
        if sum > 0:
            data['label'][i] = 'aggressive'
        else:
            data['label'][i] = 'normal'
    
    

def main():
    datasets = [
        # BMW e36 datasets 10m
        r'..\Datasets\Acceleration\Till reached distance\10m\aggressive\BMW e36\acceleration 0m 10m aggressive-23-04-2023-19-42-30.json',
        r'..\Datasets\Acceleration\Till reached distance\10m\normal\BMW e36\acceleration 0m 10m normal-23-04-2023-19-30-17.json',
        r'..\Datasets\Acceleration\Till reached distance\10m\slow\BMW e36\acceleration 0m 10m slow-23-04-2023-19-19-20.json',

    ]
    
    for dataset in datasets:
        print("Load e treino do modelo para:", dataset)

        # Carregamento dos dados específicos para classificação de velocidade
        with open(dataset, 'r') as file:
            data = load_dataset(dataset)
        classification_result = classify_acceleration(data)  # Classificar velocidade para os dados atuais
        print(classification_result)

if __name__ == "__main__":
    main()


Load e treino do modelo para: ..\Datasets\Acceleration\Till reached distance\10m\aggressive\BMW e36\acceleration 0m 10m aggressive-23-04-2023-19-42-30.json
Acceleration: 5.050509090909091, g-force: 0.51483273097952, slow threshold: 0.15444981929385598, aggressive threshold: 0.51483273097952
    accelerometerX_LeftRight  accelerometerY_FrontBack  \
0                   0.204224                  0.574472   
1                   0.230342                  0.175526   
2                   0.199742                  1.303585   
3                   0.003083                  0.753614   
4                   0.202767                  0.281722   
5                  -0.042388                 -0.066736   
6                  -0.010837                 -1.048308   
7                  -0.272801                  1.421072   
8                  -0.028238                 -0.586086   
9                  -0.077006                  0.570721   
10                 -0.093719                 -0.036977   
11          